In [6]:
from xfem import *

from solve_DG import *
from solve_CG import *
from solve_GCC import *

def Solve(method, i_t, i_s, k_t, k_t_ls, k_s, geom, gamma, struct_mesh, n_threads, solver, calc_max_dist=False):
    if method == "DG":
        return SolveDG(i_t, i_s, k_t, k_t_ls, k_s, geom, gamma=gamma, struct_mesh=struct_mesh, n_threads=n_threads, solver=solver, calc_max_dist=calc_max_dist)
    elif method == "CG":
        return SolveCG(i_t, i_s, k_t, k_t_ls, k_s, geom, gamma=gamma, struct_mesh=struct_mesh, n_threads=n_threads, solver=solver)
    elif method == "GCC":
        return SolveGCC(i_t, i_s, k_t, k_t_ls, k_s, geom, n_threads=n_threads, solver=solver)
    else:
        print("Non-valid method flag ", method)

In [7]:
def conv_study(options, special_name = ""):
    if options["refinement_strategy"] == "both":
        ref_space = True
        ref_time = True
        timei = 0
        spacei = 0
    elif options["refinement_strategy"] == "space":
        ref_space = True
        ref_time = False
        timei = options["n_ref"]+options["ref_offset"]
        spacei = 0
    elif options["refinement_strategy"] == "time":
        ref_space = False
        ref_time = True
        timei = 0
        spacei = options["n_ref"]+options["ref_offset"]

    l2errors = []
    for i in range(options["n_ref"]):
        if bool(options["cmd"]):
            l2final, l2l2er, minnzes, maxnzes, md = Solve(method = options["method"], i_t=timei, i_s=spacei, k_t = options["kt"], k_t_ls = options["ktls"],  k_s = options["ks"], geom=options["geometry"], gamma=options["gamma"], struct_mesh = options["struct_mesh"], n_threads=options["nthr"], solver=options["sol"], calc_max_dist=True)
        else:
            l2final, l2l2er, minnzes, maxnzes = Solve(method = options["method"], i_t=timei, i_s=spacei, k_t = options["kt"], k_t_ls = options["ktls"],  k_s = options["ks"], geom=options["geometry"], gamma=options["gamma"], struct_mesh = options["struct_mesh"], n_threads=options["nthr"], solver=options["sol"], calc_max_dist=False)
        print("Single run result: ", l2final, l2l2er, minnzes, maxnzes)

        if ref_space:
            spacei += 1
        if ref_time:
            timei += 1
        l2errors.append( l2l2er)
    return l2errors

In [8]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Layout
import matplotlib.pyplot as plt

In [11]:
slider_ks = widgets.IntSlider(min=1, max=6, continuous_update= False, description='ks:', value= 3)
slider_kt = widgets.IntSlider(min=1, max=6, continuous_update= False, description='kt:', value= 3)
slider_ktls = widgets.IntSlider(min=1, max=6, continuous_update= False, description='ktls:', value= 3)

slider_nref =  widgets.IntSlider(min=1, max=8, continuous_update= False, description='refs:', value= 5)
slider_gamma = widgets.FloatLogSlider(min=-10, max=5, continuous_update= False, description='gamma:', base= 10., value=0.05)

ref_strat_choice = widgets.RadioButtons(options=["both", "space", "time"],description='Ref. strategy:')
ref_offset = widgets.IntSlider(min=-5, max=5, continuous_update= False, description='ref_offset:', value= 0, disabled=True)

geom_sel = widgets.Dropdown( options=["moving_int", "moving_circle", "kite"], value="moving_int")
method_sel = widgets.Dropdown( options=["DG", "CG", "GCC"], value="DG")

controls1 = widgets.HBox( [slider_ks, slider_kt, slider_ktls])
controls2 = widgets.HBox( [slider_nref, slider_gamma])
controls3 = widgets.HBox( [ ref_strat_choice, ref_offset, widgets.VBox([ geom_sel, method_sel]) ] )

controls = widgets.VBox( [controls1, controls2, controls3])

plot_output = widgets.Output()
plot_output.clear_output()

out_printf = widgets.Output()

last_err = None

def slider_changed(change):
    global last_err
    with out_printf:
        errors = conv_study({"method": method_sel.value, "geometry": geom_sel.value, "ks": slider_ks.value, "kt": slider_kt.value, "ktls": slider_ktls.value, "refinement_strategy": ref_strat_choice.value, "n_ref": slider_nref.value, "gamma": slider_gamma.value, "struct_mesh": 1, "ref_offset" : ref_offset.value, "sol": "umfpack", "cmd": 0, "nthr" : 6})
    refnr = len(errors)
    plot_output.clear_output(wait=True)
    with plot_output:
        fig, ax = plt.subplots(figsize=(14, 8))

        if last_err != None:
            ax.plot(last_err, "-.", color='0.8', label="Last res")
        ax.plot( errors, "-o")
        ax.plot([ i for i in range(2, refnr)], [ 0.5**(2*i) for i in range(2, refnr)], "--", label="order 2", color='0.7')
        ax.plot([ i for i in range(2, refnr)], [ 0.5**(3*i) for i in range(2, refnr)], "-.", label="order 3", color='0.7')
        ax.plot([ i for i in range(2, refnr)], [ 0.5**(4*i) for i in range(2, refnr)], "--.", label="order 4", color='0.7')
        ax.plot([ i for i in range(2, refnr)], [ 0.5**(5*i) for i in range(2, refnr)], "--", label="order 5", color='0.7')
        ax.plot([ i for i in range(2, refnr)], [ 0.5**(6*i) for i in range(2, refnr)], "-.", label="order 6", color='0.7')
        ax.plot([ i for i in range(2, refnr)], [ 0.5**(7*i) for i in range(2, refnr)], "--.", label="order 7", color='0.7')
        ax.legend()
        plt.yscale("log")
        plt.ylabel(r'$| u(\dots, T) - u_h(\dots, T) |_{L^2(\Omega(T))}$')
        plt.xlabel('$i$')
        plt.show()
    if ref_strat_choice.value != "both":
        ref_offset.disabled = False
    else:
        ref_offset.disabled = True
    last_err = errors

for s in [slider_ks, slider_kt, slider_ktls, slider_nref, slider_gamma, ref_strat_choice, ref_offset, geom_sel, method_sel]:
    s.observe(slider_changed, names='value')

slider_changed({})

display(controls, plot_output)

Output()